# Politecnico di Milano
## Students: Caravano Andrea, Cantele Alberto
A.Y.: 2024-2025

Last modified: 05/04/2025
### Description: Internet of Things: Challenge n. 2 - Theoretical Exercise: Solution sketch

## Preliminary declaration

In [1]:
# Constants definition
GET_Request = 60 * 8 # Bits
GET_Response = 55 * 8 # Bits
SUBSCRIBE = 58 * 8 # Bits
SUBSCRIBE_ACK = 52 * 8 # Bits
PUBLISH = 68 * 8 # Bits
CONNECT = 54 * 8 # Bits
CONNECT_ACK = 47 * 8 # Bits
PING_REQUEST = 52 * 8 # Bits
PING_RESPONSE = 48 * 8 # Bits
Sensor_Minutes_Per_Cycle = 5 # Minutes
Valve_Minutes_Per_Cycle = 30 # Minutes
HOURS_PER_DAY = 24 # Hours
MINUTES_PER_HOUR = 60 # Minutes
E_TX = 50 * 10 ** (-9) # joule
E_RX = 58 * 10 ** (-9) # joule
E_Valve_Average_Computation= 2.4 * 10 ** (-3) # joule

In [2]:
# Preliminary computations
# Sensor:
Sensor_Cycles_Per_Hour = MINUTES_PER_HOUR / Sensor_Minutes_Per_Cycle # Cycles
Sensor_Cycles_Per_Day = Sensor_Cycles_Per_Hour * HOURS_PER_DAY # Cycles

# Valve:
Valve_Cycles_Per_Hour = MINUTES_PER_HOUR / Valve_Minutes_Per_Cycle # Cycles
Valve_Cycles_Per_Day = Valve_Cycles_Per_Hour * HOURS_PER_DAY # Cycles
E_Average_Computation = E_Valve_Average_Computation * Valve_Cycles_Per_Day

## CoAP Energy Consumption

In [3]:
# Common energy computation for CoAP
E_GetRequest_TX = GET_Request * E_TX
E_GetRequest_RX = GET_Response * E_RX
E_Sensor_GetResponse_TX = (GET_Response * E_TX) * Sensor_Cycles_Per_Day
E_Sensor_GetResponse_RX = (GET_Response * E_RX) * Sensor_Cycles_Per_Day

In [4]:
# CoAP energy computations: assuming observe pattern as available
# Temperature Sensor sequence: GETReq with observe -> GETResp -> GETResp... until all cycles in a day are executed
E_Sensor = E_GetRequest_RX + E_Sensor_GetResponse_TX # Joule

# Valve sequence: GETReq with observe -> GETResp -> GETResp... until all cycle in a day are executed
# and every 30-min compute average
E_Valve = E_GetRequest_TX + E_Sensor_GetResponse_RX + E_Average_Computation # Joule

# Total energy consumption
E_tot = E_Sensor + E_Valve # Joule

In [5]:
# Results
print("E_Sensor: %.3f mJ" % (E_Sensor * 10 ** 3))
print("E_Valve: %.3f mJ" % (E_Valve * 10 ** 3))
print("E_tot: %.3f mJ" % (E_tot * 10 ** 3))

E_Sensor: 6.362 mJ
E_Valve: 122.574 mJ
E_tot: 128.935 mJ


### Optimization proposals

1° optimization proposal:

Reduce time boundaries: a new temperature measurement is sent every 15 minutes and average is computed every 45 minutes

In [6]:
# Time boundaries optimization
Valve_Cycles_Optimized = (MINUTES_PER_HOUR / 45.0) * HOURS_PER_DAY
Sensor_Cycles_Optimized = (MINUTES_PER_HOUR / 15.0) * HOURS_PER_DAY

In [7]:
# Energy computation
E_Sensor = GET_Request * E_RX + (GET_Response * E_TX) * Sensor_Cycles_Optimized # Joule
E_Valve = GET_Request * E_TX + (GET_Response * E_RX) * Sensor_Cycles_Optimized + E_Valve_Average_Computation * Valve_Cycles_Optimized # Joule
E_tot = E_Sensor + E_Valve # Joule

# Print final results
print("E_Sensor: %.3f mJ" % (E_Sensor * 10 ** 3))
print("E_Valve: %.3f mJ" % (E_Valve * 10 ** 3))
print("E_tot: %.3f mJ" % (E_tot * 10 ** 3))

E_Sensor: 2.140 mJ
E_Valve: 79.274 mJ
E_tot: 81.414 mJ


2° optimization proposal:

Move the average calculation to the sensor so that only one message is sent every 30 minutes, instead of a block or multiple separate messages.

Assumption: the average computation on the sensor consumes the same amount of energy consumed on the valve.

In [8]:
# Energy computation
E_Sensor = GET_Request * E_RX + (GET_Response * E_TX) * Valve_Cycles_Per_Day + E_Average_Computation # Joule
E_Valve = GET_Request * E_TX + (GET_Response * E_RX) * Valve_Cycles_Per_Day # Joule
E_tot = E_Sensor + E_Valve # Joule

# Print final results
print("E_Sensor: %.3f mJ" % (E_Sensor * 10 ** 3))
print("E_Valve: %.3f mJ" % (E_Valve * 10 ** 3))
print("E_tot: %.3f mJ" % (E_tot * 10 ** 3))

E_Sensor: 116.284 mJ
E_Valve: 1.249 mJ
E_tot: 117.533 mJ


## MQTT Energy Consumption

In [9]:
# Common energy computation constants for MQTT
E_Connect_TX = CONNECT * E_TX
E_ConnectACK_RX = CONNECT_ACK * E_RX
E_Connection_Sequence = E_Connect_TX + E_ConnectACK_RX

E_Sensor_Publish_TX = (PUBLISH * E_TX) * Sensor_Cycles_Per_Day
E_Sensor_Publish_RX = (PUBLISH * E_RX) * Sensor_Cycles_Per_Day
E_Subscribe_TX = SUBSCRIBE * E_TX
E_SubscribeACK_RX = SUBSCRIBE_ACK * E_RX

# Assume that keep alive (pair of PING REQUEST and PING RESPONSE) is performed every minute
KeepAlive_Cycles_Per_Day = HOURS_PER_DAY * MINUTES_PER_HOUR
E_KeepAlive = (PING_REQUEST * E_TX + PING_RESPONSE * E_RX) * KeepAlive_Cycles_Per_Day

In [10]:
# MQTT Energy computation using Raspberry Pi as Broker
# Temperature sensor sequence: Connect -> ConnectACK -> Publish iteration every 5 minutes and every minute Ping Request is sent and acknowledged
E_Sensor = E_Connection_Sequence + E_Sensor_Publish_TX + E_KeepAlive # Joule

# Valve sequence: ConnectReq -> Connect ACK -> Subscribe -> Subscribe ACK -> Publish iteration every 5 minutes; every 30 minutes compute average and every minute Ping Request is sent and acknowledged
E_Valve = E_Connection_Sequence + E_Subscribe_TX + E_SubscribeACK_RX + E_Sensor_Publish_RX + E_Average_Computation + E_KeepAlive # Joule

# Total energy consumption
E_tot = E_Sensor + E_Valve # Joule

In [11]:
# Print results
print("E_Sensor: %.3f mJ" % (E_Sensor * 10 ** 3))
print("E_Valve: %.3f mJ" % (E_Valve * 10 ** 3))
print("E_tot: %.3f mJ" % (E_tot * 10 ** 3))

E_Sensor: 69.901 mJ
E_Valve: 186.401 mJ
E_tot: 256.302 mJ


### Optimization proposals

1° optimization proposal:

Remove liveness constraints (PING REQUEST and PING RESPONSE pair)

In [12]:
# Energy computation
E_Sensor = E_Connection_Sequence + E_Sensor_Publish_TX # Joule
E_Valve = E_Connection_Sequence + E_Subscribe_TX + E_SubscribeACK_RX + E_Sensor_Publish_RX + E_Average_Computation # Joule

# Total energy consumption
E_tot = E_Sensor + E_Valve # Joule

# Print results
print("E_Sensor: %.3f mJ" % (E_Sensor * 10 ** 3))
print("E_Valve: %.3f mJ" % (E_Valve * 10 ** 3))
print("E_tot: %.3f mJ" % (E_tot * 10 ** 3))

E_Sensor: 7.877 mJ
E_Valve: 124.378 mJ
E_tot: 132.255 mJ


1-bis optimization proposal:

Remove liveness constraints (PING REQUEST and PING RESPONSE pair)

and

Reduce time boundaries: a new temperature measurement is sent every 15 minutes and average is computed every 45 minutes

In [13]:
# Time boundaries optimization
Valve_Cycles_Optimized = (MINUTES_PER_HOUR / 45.0) * HOURS_PER_DAY
Sensor_Cycles_Optimized = (MINUTES_PER_HOUR / 15.0) * HOURS_PER_DAY

In [14]:
# Energy computation
E_Sensor = E_Connection_Sequence + (PUBLISH * E_TX) * Sensor_Cycles_Optimized # Joule
E_Valve = E_Connection_Sequence + E_Subscribe_TX + E_SubscribeACK_RX + (PUBLISH * E_RX) * Sensor_Cycles_Optimized + E_Valve_Average_Computation * Valve_Cycles_Optimized # Joule

# Total energy consumption
E_tot = E_Sensor + E_Valve # Joule

# Print results
print("E_Sensor: %.3f mJ" % (E_Sensor * 10 ** 3))
print("E_Valve: %.3f mJ" % (E_Valve * 10 ** 3))
print("E_tot: %.3f mJ" % (E_tot * 10 ** 3))

E_Sensor: 2.655 mJ
E_Valve: 79.920 mJ
E_tot: 82.574 mJ


2° optimization proposal:

A versioning system: store the result of the previously-computed average on the Raspberry Pi and publish a new message only for significative temperature variations.

In the following, a meaningful change is assumed to be present on one message over two.

In [15]:
# Set up computation
E_Broker_Store_Average = ((PUBLISH * Valve_Cycles_Per_Day) / 2 * E_RX)

In [16]:
# Energy computation
E_Sensor = E_Connection_Sequence + E_Sensor_Publish_TX + E_KeepAlive # Joule
E_Valve = E_Connection_Sequence + E_Subscribe_TX + E_SubscribeACK_RX + E_Broker_Store_Average + E_KeepAlive # Joule

E_tot = E_Sensor + E_Valve # Joule

# Print results
print("E_Sensor: %.3f mJ" % (E_Sensor * 10 ** 3))
print("E_Valve: %.3f mJ" % (E_Valve * 10 ** 3))
print("E_tot: %.3f mJ" % (E_tot * 10 ** 3))

E_Sensor: 69.901 mJ
E_Valve: 62.872 mJ
E_tot: 132.772 mJ


2-bis optimization proposal:

A versioning system: store the result of the previously-computed average on the Raspberry Pi and publish a new message only for significative temperature variations.

and

Reduce time boundaries: a new temperature measurement is sent every 15 minutes and average is computed every 45 minutes

In the following, a meaningful change is assumed to be present on one message over two.

In [17]:
# Time boundaries optimization
Valve_Cycles_Optimized = (MINUTES_PER_HOUR / 45.0) * HOURS_PER_DAY
Sensor_Cycles_Optimized = (MINUTES_PER_HOUR / 15.0) * HOURS_PER_DAY

In [18]:
# Set up computation
E_Broker_Shift_Average_Sub = ((PUBLISH * Valve_Cycles_Optimized) / 2 * E_RX)

In [19]:
# Energy computation
E_Sensor = E_Connection_Sequence + (PUBLISH * E_TX) * Sensor_Cycles_Optimized + E_KeepAlive # Joule
E_Valve = E_Connection_Sequence + E_Subscribe_TX + E_SubscribeACK_RX + E_Broker_Shift_Average_Sub + E_KeepAlive # Joule

E_tot = E_Sensor + E_Valve # Joule

# Print Results
print("E_Sensor: %.3f mJ" % (E_Sensor * 10 ** 3))
print("E_Valve: %.3f mJ" % (E_Valve * 10 ** 3))
print("E_tot: %.3f mJ" % (E_tot * 10 ** 3))

E_Sensor: 64.678 mJ
E_Valve: 62.619 mJ
E_tot: 127.298 mJ


A complete merge:

Remove liveness constraints (PING REQUEST and PING RESPONSE pair)

and

A versioning system: store the result of the previously-computed average on the Raspberry Pi and publish a new message only for significative temperature variations.

and

Reduce time boundaries: a new temperature measurement is sent every 15 minutes and average is computed every 45 minutes

In the following, a meaningful change is assumed to be present on one message over two.

In [20]:
# Time boundaries optimization
Valve_Cycles_Optimized = (MINUTES_PER_HOUR / 45.0) * HOURS_PER_DAY
Sensor_Cycles_Optimized = (MINUTES_PER_HOUR / 15.0) * HOURS_PER_DAY

In [21]:
# Set up computation
E_Broker_Shift_Average_Sub = ((PUBLISH * Valve_Cycles_Optimized) / 2 * E_RX)

In [22]:
# Energy computation
E_Sensor = E_Connection_Sequence + (PUBLISH * E_TX) * Sensor_Cycles_Optimized # Joule
E_Valve = E_Connection_Sequence + E_Subscribe_TX + E_SubscribeACK_RX + E_Broker_Shift_Average_Sub # Joule

E_tot = E_Sensor + E_Valve # Joule

# Print Results
print("E_Sensor: %.3f mJ" % (E_Sensor * 10 ** 3))
print("E_Valve: %.3f mJ" % (E_Valve * 10 ** 3))
print("E_tot: %.3f mJ" % (E_tot * 10 ** 3))

E_Sensor: 2.655 mJ
E_Valve: 0.596 mJ
E_tot: 3.250 mJ
